In [ ]:
import anvil.server
import anvil.media
from anvil.tables import app_tables

anvil.server.connect("server_key")

In [ ]:
import ffmpeg
from datetime import datetime
import os

@anvil.server.callable
def save_video_file(file):
    file_path = file.name
    if file_path.endswith('.mp4') or file_path.endswith('.avi'):
        with open(file_path, "wb") as f:
            f.write(file.get_bytes())

        if file_path.endswith('.mp4'):
            mp4_file_path = file_path
        else:
            mp4_file_path = file_path.replace('.avi', '.mp4')
            try:
                ffmpeg.input(file_path).output(mp4_file_path).run()
            except ffmpeg.Error as e:
                print("Error converting video:", e)

        with open(file_path, "rb") as f:
            avi_file_bytes = f.read()
        with open(mp4_file_path, "rb") as f:
            mp4_file_bytes = f.read()

        avi_file = anvil.BlobMedia("video/avi", avi_file_bytes, name=os.path.basename(file_path))
        mp4_file = anvil.BlobMedia("video/mp4", mp4_file_bytes, name=os.path.basename(mp4_file_path))
        existing_file = app_tables.video_files.get(file_name=file.name)
        if existing_file:
            existing_file['avi_file'] = avi_file
            existing_file['mp4_file'] = mp4_file
            existing_file['upload_date'] = datetime.now()
        else:
            app_tables.video_files.add_row(
                avi_file=avi_file,
                mp4_file=mp4_file,
                file_name=file.name,
                upload_date=datetime.now()
            )
        print(file_path)
        return file_path

    return None

In [ ]:
import cv2
import torch
import numpy as np
from ultralytics import YOLO
from tqdm import tqdm

model = YOLO('/path/to/train/weights/best.pt', verbose=True)